In [3]:
import gymnasium as gym 
from gymnasium import Env
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# Set the global seed value
global_seed = 40
np.random.seed(global_seed)
random.seed(global_seed)

class missile_interception_3d(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.MultiDiscrete([5, 5])
        self.observation_space = None
        self.np_random = np.random.RandomState(global_seed)
        self.time_step = 0.5  # seconds
        self.g = 9.81  # gravity
        self.navigation_constant = 3  # ProNav gain

        # Target box dimensions
        self.targetbox_x_min = -15000
        self.targetbox_x_max = 15000
        self.targetbox_y_min = -15000
        self.targetbox_y_max = 15000

    def generate_enemy_missile(self):
        range_min, range_max = 70000, 1000000
        self.attack_target_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.attack_target_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)

        origin_phi = self.np_random.uniform(0, 2 * np.pi)
        theta = self.np_random.uniform(0.523599, 1.0472)
        lower_limit = np.sqrt((range_min * self.g) / np.sin(2 * theta))
        upper_limit = np.sqrt((range_max * self.g) / np.sin(2 * theta))
        initial_velocity = self.np_random.uniform(lower_limit, upper_limit)
        range = initial_velocity * np.cos(theta) * (2 * initial_velocity * np.sin(theta) / self.g)

        launch_x = self.attack_target_x + range * np.cos(origin_phi)
        launch_y = self.attack_target_y + range * np.sin(origin_phi)

        if origin_phi <= (np.pi / 2):
            extra_angle = np.pi - (np.pi / 2) - origin_phi
            launch_angle = (3 * np.pi / 2) - extra_angle

        elif origin_phi <= np.pi:
            extra_angle = origin_phi - (np.pi / 2)
            launch_angle = (3 * np.pi / 2) + extra_angle

        elif origin_phi <= (3 * np.pi / 2):
            extra_angle = 3 * np.pi / 2 - origin_phi
            launch_angle = np.pi - (np.pi / 2) - extra_angle

        else:
            extra_angle = 2 * np.pi - origin_phi
            extra_angle_2 = np.pi - (np.pi / 2) - extra_angle
            launch_angle = (np.pi / 2) + extra_angle_2
            
    # Set the enemy missile’s launch parameters
        self.enemy_theta, self.enemy_azimuth = theta, launch_angle  # Launch angle replaces azimuth here
        self.enemy_initial_velocity = initial_velocity
        self.enemy_x, self.enemy_y, self.enemy_z = launch_x, launch_y, 0

    def generate_defense_missile(self):
        self.defense_launch_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.defense_launch_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)
        self.defense_azimuth = self.np_random.uniform((self.enemy_azimuth - 0.785398), (self.enemy_azimuth + 0.785398))
        self.defense_theta = 0.785398
        self.defense_initial_velocity = 3000
        self.defense_x = self.defense_launch_x
        self.defense_y = self.defense_launch_y
        self.defense_z = 0

    def move_missiles(self):
        self.defense_v0x = self.defense_initial_velocity * np.cos(self.defense_azimuth) * np.cos(self.defense_theta)
        self.defense_v0y = self.defense_initial_velocity * np.sin(self.defense_azimuth) * np.cos(self.defense_theta)
        self.defense_v0z = self.defense_initial_velocity * np.sin(self.defense_theta)

        self.defense_x += self.defense_v0x * self.time_step
        self.defense_y += self.defense_v0y * self.time_step
        self.defense_z = self.defense_v0z * self.time_step - 0.5 * self.g * self.time_step**2

        if self.enemy_z < 0:
            print("DEFENSE MISSILE HIT GROUND")
            self.done = True

        # Update enemy missile position
        self.enemy_v0x = self.enemy_initial_velocity * np.cos(self.enemy_azimuth) * np.cos(self.enemy_theta)
        self.enemy_v0y = self.enemy_initial_velocity * np.sin(self.enemy_azimuth) * np.cos(self.enemy_theta)
        self.enemy_v0z = self.enemy_initial_velocity * np.sin(self.enemy_theta)

        self.enemy_x += self.enemy_v0x * self.time_step
        self.enemy_y += self.enemy_v0y * self.time_step
        self.enemy_z = self.enemy_v0z * self.time_step - 0.5 * self.g * self.time_step**2

        if self.enemy_z < 0:
            print("ENEMY MISSILE HIT GROUND")
            self.done = True

    def calculate_pro_nav_acceleration(self):
        relative_position = np.array([self.enemy_x - self.defense_x, self.enemy_y - self.defense_y, self.enemy_z - self.defense_z])
        relative_velocity = np.array([self.enemy_v0x - self.defense_velocity[0], self.enemy_v0y - self.defense_velocity[1], self.enemy_v0z - self.defense_velocity[2]])
        range_to_target = np.linalg.norm(relative_position)

        print("Range to target: ", range_to_target)
        if range_to_target == 0:
            self.done = True
        
        # Estimate time-to-go
        closing_velocity = -np.dot(relative_velocity, relative_position) / range_to_target
        Tgo = range_to_target / closing_velocity if closing_velocity != 0 else 1  # Avoid division by zero

        # Calculate Zero Effort Miss (ZEM)
        ZEM = relative_position + relative_velocity * Tgo
        
        # Calculate ProNav acceleration
        required_acceleration = (self.navigation_constant * ZEM) / (Tgo**2)
        return required_acceleration

    def step(self, action):
        if not self.done:
            acceleration_command = self.calculate_pro_nav_acceleration()
            self.defense_velocity += acceleration_command * self.time_step
            self.move_missiles()
            self.enemy_path.append([self.enemy_x, self.enemy_y, self.enemy_z])
            self.defense_path.append([self.defense_x, self.defense_y, self.defense_z])

    def reset(self):
        self.done = False
        self.enemy_path, self.defense_path = [], []
        self.generate_enemy_missile()
        self.generate_defense_missile()
        self.defense_velocity = np.array([self.defense_initial_velocity * np.cos(self.defense_azimuth) * np.cos(self.defense_theta),
                                          self.defense_initial_velocity * np.sin(self.defense_azimuth) * np.cos(self.defense_theta),
                                          self.defense_initial_velocity * np.sin(self.defense_theta)])
        self.enemy_v0x = self.enemy_initial_velocity * np.cos(self.enemy_azimuth) * np.cos(self.enemy_theta)
        self.enemy_v0y = self.enemy_initial_velocity * np.sin(self.enemy_azimuth) * np.cos(self.enemy_theta)
        self.enemy_v0z = self.enemy_initial_velocity * np.sin(self.enemy_theta)
        self.t = 0

    def render(self):
        enemy_path_array = np.array(self.enemy_path)
        defense_path_array = np.array(self.defense_path)
        print("enemy_path_array: ", enemy_path_array)
        print("defense_path_array: ", defense_path_array)    
        xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]

        xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(xe, ye, ze, label="Enemy missile trajectory", color='blue')
        ax.plot(xd, yd, zd, label="Defense missile trajectory", color='red')
        ax.scatter(xe[0], ye[0], ze[0], color='blue', s=50, label="Enemy Start")
        ax.scatter(xd[0], yd[0], zd[0], color='red', s=50, label="Defense Start")
        ax.set_xlabel('X coordinate (m)')
        ax.set_ylabel('Y coordinate (m)')
        ax.set_zlabel('Z coordinate (m)')
        ax.set_title('Missile Trajectory')
        ax.legend()
        plt.show()

env = missile_interception_3d()
env.reset()
env.render()

# # Simulation loop
# while not env.done:
#     env.step([0, 0])  # Placeholder action, ProNav handles turning
#     env.t += env.time_step

# env.render()


UnboundLocalError: cannot access local variable 'xe' where it is not associated with a value

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML, display

def update_paths(num, xe, ye, ze, xd, yd, zd, lines, ax):
    # Update the data in the trajectory plots
    lines[0].set_data_3d(xe[:num], ye[:num], ze[:num])
    lines[1].set_data_3d(xd[:num], yd[:num], zd[:num])

    # Rotate the camera angle
    ax.view_init(elev=10, azim=1 * num)  # Rotate slowly
    return lines

def animate_trajectories(enemy_path, defense_path, num_steps):
    enemy_path_array = np.array(enemy_path)
    defense_path_array = np.array(defense_path)

    xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]
    xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim([np.min([xe, xd]), np.max([xe, xd])])
    ax.set_ylim([np.min([ye, yd]), np.max([ye, yd])])
    ax.set_zlim([np.min([ze, zd]), np.max([ze, zd])])

    # Plot initially without data
    line_enemy, = ax.plot([], [], [], 'b-', label="Enemy missile trajectory")
    line_defense, = ax.plot([], [], [], 'r-', label="Defense missile trajectory")
    lines = [line_enemy, line_defense]

    # Setting labels and titles
    ax.set_xlabel('X coordinate (m)')
    ax.set_ylabel('Y coordinate (m)')
    ax.set_zlabel('Z coordinate (m)')
    ax.set_title('Animated Missile Trajectory')
    ax.legend()

    # Creating the Animation object
    ani = animation.FuncAnimation(
        fig, update_paths, num_steps, fargs=(xe, ye, ze, xd, yd, zd, lines, ax),
        interval=20, blit=False)

    # Option 2: Display using HTML
    plt.close(fig)
    video = ani.to_html5_video()
    html = display(HTML(video))

    return html

num_steps = len(env.enemy_path)
animate_trajectories(env.enemy_path, env.defense_path, num_steps)
